**Variance Reduction via Antithetic Variates**

This notebook applies Antithetic Variate sampling to reduce variance in Monte Carlo simulations for Lookback option pricing.
By simulating mirrored Brownian motions (i.e., using both +Z and -Z random draws), we exploit symmetry in the stochastic process to reduce estimator noise and improve convergence, achieving more accurate results with fewer simulations.

In [1]:
import numpy as np
from scipy.stats import norm

# --- Antithetic Path Simulation ---
def antithetic_mc_sim(S0, sigma, T, r, num_sim, num_steps):
    dt = T / num_steps
    Z = np.random.randn(num_sim, num_steps)
    paths_pos = np.zeros((num_sim, num_steps))
    paths_neg = np.zeros((num_sim, num_steps))
    
    for i in range(num_sim):
        paths_pos[i, 0] = paths_neg[i, 0] = S0
        for j in range(1, num_steps):
            drift = (r - 0.5*sigma**2)*dt
            diffusion = sigma*np.sqrt(dt)
            paths_pos[i, j] = paths_pos[i, j-1] * np.exp(drift + diffusion*Z[i, j])
            paths_neg[i, j] = paths_neg[i, j-1] * np.exp(drift - diffusion*Z[i, j])
    return paths_pos, paths_neg

# --- Lookback Pricing with Antithetic Variates ---
def lookback_antithetic(S0, sigma, T, r, num_sim, num_steps):
    pos, neg = antithetic_mc_sim(S0, sigma, T, r, num_sim, num_steps)
    payoffs_pos = [pos[i, -1] - np.min(pos[i]) for i in range(num_sim)]
    payoffs_neg = [neg[i, -1] - np.min(neg[i]) for i in range(num_sim)]
    combined = 0.5 * (np.array(payoffs_pos) + np.array(payoffs_neg))
    return np.mean(combined), np.std(combined)

# --- Simulation Parameters ---
S0, sigma, T, r = 80, 0.1, 1, 0.05
num_sim, num_steps = 500, 100

mean_lb, std_lb = lookback_antithetic(S0, sigma, T, r, num_sim, num_steps)
z_score = norm.ppf(0.99875)
margin = z_score * std_lb / np.sqrt(num_sim)
print(f"99.75% CI: [{mean_lb - margin:.4f}, {mean_lb + margin:.4f}]")


99.75% CI: [7.9434, 8.5496]
